# HW 4
## Prepare notebook

In [3]:
%pip install pandas
%pip install scikit-learn
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached openpyxl-3.1.2-py2.py3-none-any.whl (249 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel("DataOnly/WaitData.Published.xlsx")
print(df.shape)
df.head()

(23583, 86)


,Wait,SumHowEarlyWaiting,AvgHowEarlyWaiting,LineCount0Strict,SumWaits,LineCount0,LineCount1,LineCount2,LineCount3,LineCount4,...,NumAddOnsToday,NumAddOnsLastW2,NumScheduledNextSlot,NumScheduledNextW2,SumTimeToCompleteNextSlot,SumTimeToCompleteNextW2,WithContrastCountWaiting,WithAndWithoutContrastCountWaiting,WithContrastCountInProgress,WithAndWithoutContrastCountInProgress
0,0,372,62,6,249,6,5,6,6,4,...,4,2,0,0,0,0,3,1,1,0
1,-3,335,67,3,195,5,4,6,6,5,...,4,1,2,0,32,0,2,1,2,0
2,-25,274,68,4,150,4,5,6,4,6,...,4,1,1,1,13,16,1,1,3,0
3,36,369,73,2,179,5,6,4,5,5,...,3,1,2,1,45,16,3,1,1,0
4,-21,438,73,5,192,6,5,5,6,5,...,3,0,2,1,32,15,4,0,0,1


In [3]:
X = df.drop('Wait', axis=1)
Y = df['Wait']
print(X.shape, Y.shape)

(23583, 85) (23583,)


## Task 5

In [4]:
from sklearn import linear_model
model = linear_model.LinearRegression()
model.fit(X, Y)
Ypred = model.predict(X) # use trained regression model to predict
r = Y - Ypred # compute prediction error (residual)
e = abs(r).mean() # compute model error
print(f"Model error: {e}")


Model error: 23.16628893634778


## Task 6

In [7]:
# Run Python feature selection
print('\n>Python feature selection:')
from sklearn.feature_selection import RFE
from itertools import compress

for nFeatures in range(1,4):
    rfe = RFE(model, n_features_to_select=nFeatures)
    X_rfe = rfe.fit_transform(X,Y) # transforming data using RFE   

    model.fit(X_rfe,Y)
    cols = list(compress(X.columns, rfe.support_))
    model.fit(X[cols], Y)
    e = abs(Y - model.predict(X[cols])).mean()
    print(e, cols)



>Python feature selection:
32.50541172120655 ['CardiacCount']
31.224937706982736 ['CardiacCount', 'DelayedInLine']
31.25752894868932 ['CardiacCount', 'StartTime', 'DelayedInLine']


## Task 7 - 8

In [9]:
def feature_selection(n_features):
    best_features = []
    for i in range(1, n_features + 1):
        errors = []
        for col in X.columns:
            model = linear_model.LinearRegression()
            model.fit(X[best_features + [col]], Y)
            Ypred = model.predict(X[best_features + [col]])  
            e = abs(Y - Ypred).mean() 
            errors.append((col, e))
        errors = sorted(errors, key=lambda x: x[1])
        best_features.append(errors[0][0])
        print(f'{i} feature', errors[0])

In [10]:
feature_selection(3)

1 feature ('DelayedInLine', 31.227057699733514)
2 feature ('LineCount0Strict', 30.137965863467567)
3 feature ('LineCount0', 28.69649063225338)


In [11]:
feature_selection(15)

1 feature ('DelayedInLine', 31.227057699733514)
2 feature ('LineCount0Strict', 30.137965863467567)
3 feature ('LineCount0', 28.69649063225338)
4 feature ('SumDelayWaitingInLine', 28.246309585132312)
5 feature ('AheadCount', 28.147006842164522)
6 feature ('NumCompletedToday', 24.98957473477478)
7 feature ('NumScheduledNextW2', 24.65918640725134)
8 feature ('DelayCount', 24.329071006187288)
9 feature ('InProgressSize', 24.071936786758197)
10 feature ('StartTime4', 23.846509492065223)
11 feature ('AvgDelayForDay', 23.69261939854719)
12 feature ('AvgWaitLastK1Customers', 23.62791429658231)
13 feature ('SumDelayInProgress', 23.579159670479648)
14 feature ('NumScheduledNextSlot', 23.54808179388178)
15 feature ('IsFirst', 23.52096351255335)


## Task 9

In [12]:
import itertools
n_features = 3
errors = []
for i in range(1, n_features + 1):
    for subset in itertools.combinations(X.columns, i):
        model = linear_model.LinearRegression()
        model.fit(X[list(subset)], Y)
        Ypred = model.predict(X[list(subset)]) 
        e = abs(Y - Ypred).mean() 
        errors.append((subset, e))
errors = sorted(errors, key=lambda x: x[1])
print(errors[0])

(('AheadCount', 'SumDelayWaitingByExamCode', 'NumCompletedToday'), 26.072745167091245)
